In [3]:
# %%
## Memorializing this as the easy way to lookup accounts


# %%
"""
Main Entry Point
"""
from pathlib import Path
from typing import List
import argparse
from datetime import datetime

import pandas as pd # type: ignore

import src.additional_fields
import src.fetch_data # type: ignore
import src.core_transform # type: ignore
import cdutils.pkey_sqlite # type: ignore
import cdutils.hhnbr # type: ignore
import cdutils.loans.calculations
import cdutils.inactive_date
import cdutils.input_cleansing

# Current (doesn't really work without)
# data = src.fetch_data.fetch_data()


# # Specific date
# specified_date = datetime(2020, 12, 31)

def query_df_on_date(specified_date):

    data = src.fetch_data.fetch_data(specified_date)

    # # # Core transformation pipeline
    raw_data = src.core_transform.main_pipeline(data)

    # Raw data with pkey appended
    raw_data = cdutils.pkey_sqlite.add_pkey(raw_data)
    raw_data = cdutils.pkey_sqlite.add_ownership_key(raw_data)
    raw_data = cdutils.pkey_sqlite.add_address_key(raw_data)

    # %%
    househldacct = data['househldacct'].copy()
    raw_data = cdutils.hhnbr.add_hh_nbr(raw_data, househldacct)

    # %%
    raw_data

    # %%

    # %%
    loan_category_df = cdutils.loans.calculations.categorize_loans(raw_data)

    # %%
    loan_category_df = loan_category_df[['acctnbr','Category']].copy()

    # %%
    df = pd.merge(raw_data, loan_category_df, on='acctnbr', how='left')

    # %%
    df

    # %%

    df = cdutils.inactive_date.append_inactive_date(df)

    # %%
    df

    # %%
    pers = data['wh_pers'].copy()

    # %%
    df.info()

    # %%

    additional_fields = src.additional_fields.fetch_data(specified_date)

    # %%
    additional_fields_to_append = additional_fields['wh_acctcommon'].copy()

    # %%

    additional_fields_to_append_schema = {
        'acctnbr':'str'
    }

    additional_fields_to_append = cdutils.input_cleansing.enforce_schema(additional_fields_to_append, additional_fields_to_append_schema)

    # %%
    df = pd.merge(df, additional_fields_to_append, how='left', on='acctnbr')

    # %%
    return df

# %%
# OUTPUT_PATH = Path('acct_table.csv')
# df.to_csv(OUTPUT_PATH, index=False)

# %%
# import hashlib

# %%
# def mask_pii(data, columns_to_mask, length=10):
#     """
#     Create a masking layer

#     Pass in a dataframe to blackbox abstraction and get a dataframe returned with masked PII in specified fields

#     Parameters:
#     - data: raw data
#     - columns_to_mask: list of columns
#     - length: length of hash (10+ is recommended based on size of the data)
#     """
#     df_hashed = data.copy()

#     for col in columns_to_mask:
#         if col in df_hashed.columns:
#             df_hashed[col] = df_hashed[col].astype(str).apply(
#                 lambda x: hashlib.sha256(x.encode('utf-8')).hexdigest()[:length]
#             )
#         else:
#             raise ValueError(f"Column {col} not found in dataframe passed in")
        
#     return df_hashed

# %%
# columns_to_mask = [
#     'acctnbr',
#     'ownersortname',
#     'loanofficer',
#     'acctofficer',
#     'taxrptfororgnbr',
#     'taxrptforpersnbr'
#     ]

# masked_df = mask_pii(df, columns_to_mask)

# %%
# masked_df

# %%
# transposed_df = masked_df.head(3).T.reset_index()

# %%
# transposed_df

# %%
# OUTPUT_PATH = Path('masked_acct_table.csv')
# masked_df.to_csv(OUTPUT_PATH, index=False)

# %%





In [4]:
specified_date = datetime(2020, 12, 31)
data2020 = query_df_on_date(specified_date)
data2020

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 91019 entries, 0 to 91018
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   effdate                 91019 non-null  datetime64[ns]
 1   acctnbr                 91019 non-null  object        
 2   ownersortname           91019 non-null  object        
 3   product                 91019 non-null  object        
 4   noteopenamt             91019 non-null  float64       
 5   ratetypcd               72634 non-null  object        
 6   mjaccttypcd             91019 non-null  object        
 7   currmiaccttypcd         91019 non-null  object        
 8   curracctstatcd          91019 non-null  object        
 9   noteintrate             91019 non-null  float64       
 10  bookbalance             91019 non-null  float64       
 11  notebal                 91019 non-null  float64       
 12  contractdate            91019 non-null  dateti

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate,branchname,primaryownercity,primaryownerstate
0,2020-12-31,150200005,MoneyGram Payment Systems Inc,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,41516.0,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN,NaT,BCSB - Main Office,Minneapolis,MN
1,2020-12-31,61053112,City Of Taunton,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.001500,...,4183.0,33668.0,33967.0,97724.0,2020-02-26 23:28:03,NaN,NaT,BCSB - Muni Main Office,Taunton,MA
2,2020-12-31,150544263,REDBROOK APARTMENTS LLC,Commercial Mortgages,42000000.0,VAR,CML,CM40,ACT,0.030000,...,57113.0,59661.0,61147.0,193906.0,2021-12-07 13:23:49,CRE,NaT,BCSB - Comm'l Lending- Taunton,Braintree,MA
3,2020-12-31,60436522,Hodess Construction Corp,Investment Business Checking,0.0,VAR,CK,CK28,ACT,0.002000,...,3315.0,3942.0,4388.0,241524.0,2022-09-08 22:14:08,NaN,NaT,BCSB - No Attleboro Branch,North Attleboro,MA
4,2020-12-31,6252261,Hutchens Holding II LLC,Commercial Swap Mortgage,13405500.0,VAR,CML,CM43,ACT,0.026586,...,3996.0,4209.0,4432.0,230789.0,2024-07-29 12:35:49,CRE,NaT,BCSB - Comm'l Lending - Candleworks,Fall River,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91014,2020-12-31,150291616,"Fisher, Aquinnah C.",eChecking 16/17,100.0,None,CK,CK03,ACT,0.000000,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,BCSB - Walmart Branch,Taunton,MA
91015,2020-12-31,150513086,"Brooks, Cadeem C.",Basic Checking,50.0,None,CK,CK02,ACT,0.000000,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,BCSB - North Raynham Branch,Lowell,MA
91016,2020-12-31,27039064,Bristol County Savings Bank,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,1221.0,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - Main Office,Taunton,MA
91017,2020-12-31,150337783,Bristol County Savings Bank,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,1221.0,1352.0,51094.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - Contact Center,Taunton,MA


In [5]:
specified_date = datetime(2021, 12, 31)
data2021 = query_df_on_date(specified_date)
data2021

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88944 entries, 0 to 88943
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   effdate                 88944 non-null  datetime64[ns]
 1   acctnbr                 88944 non-null  object        
 2   ownersortname           88944 non-null  object        
 3   product                 88944 non-null  object        
 4   noteopenamt             88944 non-null  float64       
 5   ratetypcd               70453 non-null  object        
 6   mjaccttypcd             88944 non-null  object        
 7   currmiaccttypcd         88944 non-null  object        
 8   curracctstatcd          88944 non-null  object        
 9   noteintrate             88944 non-null  float64       
 10  bookbalance             88944 non-null  float64       
 11  notebal                 88944 non-null  float64       
 12  contractdate            88944 non-null  dateti

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate,branchname,primaryownercity,primaryownerstate
0,2021-12-31,150200005,MoneyGram Payment Systems Inc,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.0000,...,41516.0,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN,NaT,BCSB - Main Office,Minneapolis,MN
1,2021-12-31,61053112,City Of Taunton,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.0010,...,4183.0,33668.0,33967.0,97724.0,2020-02-26 23:28:03,NaN,NaT,BCSB - Muni Main Office,Taunton,MA
2,2021-12-31,60436522,Hodess Construction Corporation,Investment Business Checking,0.0,VAR,CK,CK28,ACT,0.0015,...,3315.0,3942.0,4388.0,241524.0,2022-09-08 22:14:08,NaN,NaT,BCSB - No Attleboro Branch,North Attleboro,MA
3,2021-12-31,4426080386,"Barry, Richard L.",Statement Savings,0.0,VAR,SAV,SV02,ACT,0.0005,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,BCSB - No Attleboro Branch,ATTLEBORO,MA
4,2021-12-31,60032826,City Of Attleboro,Municipal Now,0.0,VAR,CK,CK27,ACT,0.0003,...,5106.0,35118.0,35620.0,206654.0,2024-04-29 22:10:12,NaN,NaT,BCSB - Muni Attleboro Branch,Attleboro,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88939,2021-12-31,27059932,Willow Tree Poultry Farm Inc,Business Checking,0.0,None,CK,CK12,ACT,0.0000,...,33010.0,33626.0,33919.0,191089.0,2020-02-26 23:28:03,NaN,NaT,BCSB - Attleboro Branch,Attleboro,MA
88940,2021-12-31,150569667,Bristol County Savings Bank,Business Checking,0.0,None,CK,CK12,ACT,0.0000,...,1221.0,1352.0,62702.0,NaN,NaT,NaN,NaT,BCSB - Deposit Operations,Taunton,MA
88941,2021-12-31,27039064,Bristol County Savings Bank,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.0000,...,1221.0,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - Main Office,Taunton,MA
88942,2021-12-31,150337783,Bristol County Savings Bank,Business Checking,0.0,None,CK,CK12,ACT,0.0000,...,1221.0,1352.0,51094.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - Contact Center,Taunton,MA


In [7]:
specified_date = datetime(2022, 12, 30)
data2022 = query_df_on_date(specified_date)
data2022

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88835 entries, 0 to 88834
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   effdate                 88835 non-null  datetime64[ns]
 1   acctnbr                 88835 non-null  object        
 2   ownersortname           88835 non-null  object        
 3   product                 88835 non-null  object        
 4   noteopenamt             88835 non-null  float64       
 5   ratetypcd               70564 non-null  object        
 6   mjaccttypcd             88835 non-null  object        
 7   currmiaccttypcd         88835 non-null  object        
 8   curracctstatcd          88835 non-null  object        
 9   noteintrate             88835 non-null  float64       
 10  bookbalance             88835 non-null  float64       
 11  notebal                 88835 non-null  float64       
 12  contractdate            88835 non-null  dateti

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate,branchname,primaryownercity,primaryownerstate
0,2022-12-30,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.0000,...,41516.0,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN,NaT,BCSB - MAIN OFFICE,MINNEAPOLIS,MN
1,2022-12-30,150833377,"Alice Building, LLC",Commercial Mortgages,37275000.0,FIX,CML,CM40,ACT,0.0575,...,71066.0,74531.0,76376.0,260303.0,2022-12-23 22:25:51,CRE,NaT,BCSB - COMM'L LENDING- TAUNTON,Providence,RI
2,2022-12-30,60032826,CITY OF ATTLEBORO,Municipal Now,0.0,VAR,CK,CK27,ACT,0.0003,...,5106.0,35118.0,35620.0,206654.0,2024-04-29 22:10:12,NaN,NaT,BCSB - MUNI ATTLEBORO BRANCH,ATTLEBORO,MA
3,2022-12-30,150586025,GREATER FALL RIVER V.S.D,Municipal Now,0.0,VAR,CK,CK27,ACT,0.0003,...,40677.0,42158.0,43236.0,185474.0,2021-03-25 22:19:28,NaN,NaT,BCSB - MUNI FALL RIVER BRANCH,FALL RIVER,MA
4,2022-12-30,150809211,29 CENTER STREET LLC,Commercial Mortgages,18000000.0,FIX,CML,CM40,ACT,0.0490,...,3864.0,48607.0,75391.0,229910.0,2022-11-03 14:12:28,CRE,NaT,BCSB - COMM'L LENDING- TAUNTON,Taunton,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88830,2022-12-30,150818907,"BURY, CAROL A.",eChecking (18 & over),10.0,None,CK,CK04,ACT,0.0000,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,BCSB - MAIN OFFICE,East Taunton,MA
88831,2022-12-30,150569667,BRISTOL COUNTY SAVINGS BANK,Business Checking,0.0,None,CK,CK12,DORM,0.0000,...,1221.0,1352.0,62702.0,NaN,NaT,NaN,NaT,BCSB - DEPOSIT OPERATIONS,TAUNTON,MA
88832,2022-12-30,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.0000,...,1221.0,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - MAIN OFFICE,TAUNTON,MA
88833,2022-12-30,150337783,BRISTOL COUNTY SAVINGS BANK,Business Checking,0.0,None,CK,CK12,ACT,0.0000,...,1221.0,1352.0,51094.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - CONTACT CENTER,TAUNTON,MA


In [8]:
specified_date = datetime(2023, 12, 29)
data2023 = query_df_on_date(specified_date)
data2023

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90750 entries, 0 to 90749
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   effdate                 90750 non-null  datetime64[ns]
 1   acctnbr                 90750 non-null  object        
 2   ownersortname           90750 non-null  object        
 3   product                 90750 non-null  object        
 4   noteopenamt             90750 non-null  float64       
 5   ratetypcd               72775 non-null  object        
 6   mjaccttypcd             90750 non-null  object        
 7   currmiaccttypcd         90750 non-null  object        
 8   curracctstatcd          90750 non-null  object        
 9   noteintrate             90750 non-null  float64       
 10  bookbalance             90750 non-null  float64       
 11  notebal                 90750 non-null  float64       
 12  contractdate            90750 non-null  dateti

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate,branchname,primaryownercity,primaryownerstate
0,2023-12-29,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,41516.0,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN,NaT,BCSB - MAIN OFFICE,MINNEAPOLIS,MN
1,2023-12-29,150847394,COUNTY OF BRISTOL,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.037500,...,4205.0,4427.0,4671.0,NaN,NaT,NaN,NaT,BCSB - MUNI MAIN OFFICE,TAUNTON,MA
2,2023-12-29,150887241,CITY OF TAUNTON,ICS Shadow - Muni,0.0,None,CK,CK36,ACT,0.000000,...,4183.0,33668.0,33967.0,NaN,NaT,NaN,NaT,BCSB - MUNI MAIN OFFICE,TAUNTON,MA
3,2023-12-29,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.075959,...,57113.0,59661.0,61147.0,193906.0,2023-09-22 08:01:38,CRE,NaT,BCSB - COMM'L LENDING- TAUNTON,BRAINTREE,MA
4,2023-12-29,150847401,COUNTY OF BRISTOL,15 Month Municipal CD,0.0,FIX,TD,CD85,ACT,0.042500,...,4205.0,4427.0,4671.0,NaN,NaT,NaN,NaT,BCSB - MUNI MAIN OFFICE,TAUNTON,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90745,2023-12-29,150713503,"CAPPO MANAGEMENT LII, LLC",Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,65487.0,68575.0,70249.0,241811.0,2022-02-25 22:36:05,NaN,NaT,BCSB - NORTH RAYNHAM BRANCH,BROCKTON,MA
90746,2023-12-29,27047636,BRISTOL COUNTY SAVINGS BANK,Payoff Overages,0.0,None,BKCK,BTC2,ACT,0.000000,...,1221.0,1352.0,41645.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - MAIN OFFICE,TAUNTON,MA
90747,2023-12-29,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,1221.0,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - MAIN OFFICE,TAUNTON,MA
90748,2023-12-29,150337783,BRISTOL COUNTY SAVINGS BANK,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,1221.0,1352.0,51094.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - MAIN OFFICE,TAUNTON,MA


In [9]:
specified_date = datetime(2024, 12, 31)
data2024 = query_df_on_date(specified_date)
data2024

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90812 entries, 0 to 90811
Data columns (total 47 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   effdate                 90812 non-null  datetime64[ns]
 1   acctnbr                 90812 non-null  object        
 2   ownersortname           90812 non-null  object        
 3   product                 90812 non-null  object        
 4   noteopenamt             90812 non-null  float64       
 5   ratetypcd               72843 non-null  object        
 6   mjaccttypcd             90812 non-null  object        
 7   currmiaccttypcd         90812 non-null  object        
 8   curracctstatcd          90812 non-null  object        
 9   noteintrate             90812 non-null  float64       
 10  bookbalance             90812 non-null  float64       
 11  notebal                 90812 non-null  float64       
 12  contractdate            90811 non-null  dateti

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate,branchname,primaryownercity,primaryownerstate
0,2024-12-31,150200005,MONEYGRAM PAYMENT SYSTEMS INC,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,41516,43261,44260,187340.0,2020-02-26 23:28:03,NaN,NaT,BCSB - MAIN OFFICE,MINNEAPOLIS,MN
1,2024-12-31,150586025,GREATER FALL RIVER V.S.D,Municipal Now,0.0,VAR,CK,CK27,ACT,0.000300,...,40677,42158,43236,185474.0,2021-03-25 22:19:28,NaN,NaT,BCSB - MUNI FALL RIVER BRANCH,FALL RIVER,MA
2,2024-12-31,150936915,REDBROOK APARTMENTS LLC,Commercial Mortgages,65500000.0,VAR,CML,CM40,ACT,0.067809,...,57113,59661,61147,193906.0,2023-09-22 08:01:38,CRE,NaT,BCSB - COMM'L LENDING- TAUNTON,BRAINTREE,MA
3,2024-12-31,150847394,COUNTY OF BRISTOL,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.035000,...,4205,4427,4671,NaN,NaT,NaN,NaT,BCSB - MUNI MAIN OFFICE,TAUNTON,MA
4,2024-12-31,151058057,NBPIV SARATOGA LLC,Commercial Mortgages,26000000.0,FIX,CML,CM40,ACT,0.063500,...,79217,83546,85431,288455.0,2024-06-11 12:02:04,CRE,NaT,BCSB - COMM'L LENDING- TAUNTON,WAKEFIELD,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90807,2024-12-31,27047237,FIRST AUTOMOTIVE GROUP INC,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,3730,3935,41292,188829.0,2023-10-19 11:11:11,NaN,NaT,BCSB - MAIN OFFICE,NORTH ATTLEBORO,MA
90808,2024-12-31,151116235,FTRF 143 INC,Simple Business Checking,500.0,None,CK,CK25,ACT,0.000000,...,81399,85988,87850,296485.0,2024-12-16 20:42:14,NaN,NaT,BCSB - CANDLEWORKS BRANCH,EAST WAREHAM,MA
90809,2024-12-31,27039064,BRISTOL COUNTY SAVINGS BANK,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,1221,1352,41577,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - MAIN OFFICE,TAUNTON,MA
90810,2024-12-31,27020568,BRISTOL COUNTY SAVINGS BANK,PDO Checks - OLD,0.0,None,BKCK,BTC1,ACT,0.000000,...,1221,1352,41437,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - MAIN OFFICE,TAUNTON,MA


In [21]:
def get_deposit_totals(df):
    df = df[df['mjaccttypcd'].isin(['CK','SAV','TD'])].copy()
    sum = df['Net Balance'].sum()
    return sum

In [22]:
sum2020 = get_deposit_totals(data2020) 

In [ ]:
f"{sum2020:,.2f}"

'2,283,515,795.77'

In [ ]:
sum2021 = get_deposit_totals(data2021)


In [ ]:
sum2022 = get_deposit_totals(data2022)
sum2023 = get_deposit_totals(data2023)
sum2024 = get_deposit_totals(data2024)


In [29]:
print(f"2020: ${sum2020:,.2f}")
print(f"2021: ${sum2021:,.2f}")
print(f"2022: ${sum2022:,.2f}")
print(f"2023: ${sum2023:,.2f}")
print(f"2024: ${sum2024:,.2f}")


2020: $2,283,515,795.77
2021: $2,513,328,664.16
2022: $2,400,121,111.40
2023: $2,451,179,978.56
2024: $2,509,165,954.44


In [30]:
data2020

,effdate,acctnbr,ownersortname,product,noteopenamt,ratetypcd,mjaccttypcd,currmiaccttypcd,curracctstatcd,noteintrate,...,portfolio_key,ownership_key,address_key,householdnbr,datelastmaint,Category,inactivedate,branchname,primaryownercity,primaryownerstate
0,2020-12-31,150200005,MoneyGram Payment Systems Inc,Treasurer's Check,0.0,None,BKCK,TRCK,ACT,0.000000,...,41516.0,43261.0,44260.0,187340.0,2020-02-26 23:28:03,NaN,NaT,BCSB - Main Office,Minneapolis,MN
1,2020-12-31,61053112,City Of Taunton,Municipal Money Market,0.0,VAR,CK,CK18,ACT,0.001500,...,4183.0,33668.0,33967.0,97724.0,2020-02-26 23:28:03,NaN,NaT,BCSB - Muni Main Office,Taunton,MA
2,2020-12-31,150544263,REDBROOK APARTMENTS LLC,Commercial Mortgages,42000000.0,VAR,CML,CM40,ACT,0.030000,...,57113.0,59661.0,61147.0,193906.0,2021-12-07 13:23:49,CRE,NaT,BCSB - Comm'l Lending- Taunton,Braintree,MA
3,2020-12-31,60436522,Hodess Construction Corp,Investment Business Checking,0.0,VAR,CK,CK28,ACT,0.002000,...,3315.0,3942.0,4388.0,241524.0,2022-09-08 22:14:08,NaN,NaT,BCSB - No Attleboro Branch,North Attleboro,MA
4,2020-12-31,6252261,Hutchens Holding II LLC,Commercial Swap Mortgage,13405500.0,VAR,CML,CM43,ACT,0.026586,...,3996.0,4209.0,4432.0,230789.0,2024-07-29 12:35:49,CRE,NaT,BCSB - Comm'l Lending - Candleworks,Fall River,MA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91014,2020-12-31,150291616,"Fisher, Aquinnah C.",eChecking 16/17,100.0,None,CK,CK03,ACT,0.000000,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,BCSB - Walmart Branch,Taunton,MA
91015,2020-12-31,150513086,"Brooks, Cadeem C.",Basic Checking,50.0,None,CK,CK02,ACT,0.000000,...,NaN,NaN,NaN,NaN,NaT,NaN,NaT,BCSB - North Raynham Branch,Lowell,MA
91016,2020-12-31,27039064,Bristol County Savings Bank,Escrow Checks Processing,0.0,None,BKCK,BTC3,ACT,0.000000,...,1221.0,1352.0,41577.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - Main Office,Taunton,MA
91017,2020-12-31,150337783,Bristol County Savings Bank,Business Checking,0.0,None,CK,CK12,ACT,0.000000,...,1221.0,1352.0,51094.0,255924.0,2022-09-27 22:46:27,NaN,NaT,BCSB - Contact Center,Taunton,MA
